In [ ]:
# Copyright (c) 2020 ZZH

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, MaxPool2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
import os
import numpy as np
import math

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
epochs = 100
lr = 0.05
batch_size = 32
REGULARIZER  = 0.0001
checkpoint_save_path =  './Model/SqueezeNet/'
log_dir = os.path.join("Model","SqueezeNet_logs")

In [ ]:
#数据导入及数据增强
cifar10 = tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
mean = [125.307, 122.95, 113.865]  #np.mean()
std = [62.9932, 62.0887, 66.7048]  #np.std()
for i in range(3):
    x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
    x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

DataGenTrain = tf.keras.preprocessing.image.ImageDataGenerator(
               rotation_range = 15,
               width_shift_range = 0.1,
               height_shift_range = 0.1,
               horizontal_flip = True,
               vertical_flip = False,
               shear_range=0.1,
               zoom_range = 0.1)
DataGenTrain.fit(x_train)

In [ ]:
def scheduler(epoch):  #HTD(-6,3)衰减
    start = -6.0
    end = 3.0
    if epoch < 5:
        return 0.01 * epoch + 0.001
    return lr / 2.0 * (1- math.tanh( (end-start)*epoch/epochs + start))

In [ ]:
class FireBlock(Model):
    def __init__(self,channels,bypass):
        super(FireBlock,self).__init__()
        self.bypass = bypass
        self.s1 = Conv2D(filters=channels, kernel_size=1, strides=1, padding='same', activation='relu',
                         kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.e1 = Conv2D(filters=4 * channels, kernel_size=1, strides=1, padding='same', activation='relu',
                         kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.e2 = Conv2D(filters=4 * channels, kernel_size=3, strides=1, padding='same', activation='relu',
                         kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
    def call(self,inputs):
        x = self.s1(inputs)
        x1 = self.e1(x)
        x2 = self.e2(x)
        outputs = tf.concat([x1,x2],axis = -1)
        if self.bypass:
            outputs += inputs
        return outputs

In [ ]:
class SqueezeNet(Model):
    def __init__(self):
        super(SqueezeNet,self).__init__()
        self.channels = [16,32,48,64]
        self.c1 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu',
                         kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.p1 = MaxPool2D(pool_size=2, strides=2, padding='valid')
        
        self.blocks = Sequential()
        self.blocks.add(FireBlock(channels=self.channels[0], bypass=False))
        self.blocks.add(FireBlock(channels=self.channels[0], bypass=True))
        self.blocks.add(MaxPool2D(pool_size=2, strides=2, padding='valid'))
        self.blocks.add(FireBlock(channels=self.channels[1], bypass=False))
        self.blocks.add(FireBlock(channels=self.channels[1], bypass=True))
        self.blocks.add(MaxPool2D(pool_size=2, strides=2, padding='valid'))
        self.blocks.add(FireBlock(channels=self.channels[2], bypass=False))
        self.blocks.add(FireBlock(channels=self.channels[2], bypass=True))
        self.blocks.add(FireBlock(channels=self.channels[3], bypass=False))
        self.blocks.add(FireBlock(channels=self.channels[3], bypass=True))
        
        self.d1 = Dropout(0.5)
        self.c2 = Conv2D(filters=10, kernel_size=1, strides=1, padding='same', activation='relu',
                         kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.p2 = GlobalAveragePooling2D()
        self.a1 = Activation('softmax')
        
    def call(self,inputs):
        x = self.c1(inputs)
        x = self.p1(x)
        x = self.blocks(x)
        x = self.d1(x)
        x = self.c2(x)
        x = self.p2(x)
        y = self.a1(x)
        return y

In [ ]:
model = SqueezeNet()

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True, clipnorm=2.),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              metrics=['accuracy'])

callbacks = [
            tf.keras.callbacks.LearningRateScheduler(scheduler),  #学习率衰减表
            #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_lr=0.0001, patience=10, cooldown=0)
            tf.keras.callbacks.ModelCheckpoint(     #模型保存
                filepath = checkpoint_save_path,
                save_weights_only = False,
                monitor = 'val_accuracy',
                save_best_only = True),
#             tf.keras.callbacks.EarlyStopping(       #早停
#                 monitor = 'val_accuracy',
#                 patience=15, 
#                 baseline=None),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=False)  #保存计算图
]

hist = model.fit(DataGenTrain.flow(x_train,y_train,batch_size=batch_size,shuffle=True),
                 epochs=epochs,
                 validation_data=(x_test,y_test),
                 validation_freq=1,
                 callbacks=callbacks)

model.summary()

In [ ]:
#结果可视化
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use({'figure.figsize':(6,4)})

plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
#tensorboard可视化
#!tensorboard --logdir=./Model/SqueezeNet_logs
#http://localhost:6006/

In [ ]:
print('best result: {:.2f}%  ({}epochs)'.format(100*max(hist.history['val_accuracy']),1+hist.history['val_accuracy'].index(max(hist.history['val_accuracy']))))
# best result: 88.41%  (96epochs)